In [129]:
from datasets import load_dataset

dataset = load_dataset("PaulAdversarial/all_news_finance_sm_1h2023")

In [131]:
import pandas as pd 

In [132]:
df = pd.DataFrame.from_dict(dataset["train"])

In [134]:
df = df[df['description'].str.contains('Bitcoin')]


In [ ]:
df['created_at'] = df['created_at'].str.replace('[a-zA-Z]', '', regex=True)
df['created_at'] = df['created_at'].str.slice(stop=-12)



In [137]:
import requests
from datetime import datetime

# Remplacez 'YOUR_API_KEY' et 'YOUR_SECRET_KEY' par votre clé API et votre clé secrète
api_key = 'nFHGRxf1R6RU0cjMvehIaUGKpBzwxRNcygPNgvvUMftaOJWjpB12cbcfO9s11n87'
api_secret = 'FXlPX13GMkLmrW34L3PHefeoNgxrVD4eiXFIh4Li7E2MIqMwfWU4yZ1nZS7fc3t2'

# URL de l'API de Binance pour obtenir les données de marché (24 heures de données par heure)
url = 'https://api.binance.com/api/v3/klines'
params = {
    'symbol': 'BTCUSDT',  # Le symbole du Bitcoin par rapport au dollar US
    'interval': '1d',     # Intervalle de temps (1h pour obtenir les données heure par heure)
    'limit': 1000           # Nombre de données à récupérer (24 heures)
}

# Faites une requête GET à l'API de Binance
response = requests.get(url, params=params, headers={'X-MBX-APIKEY': api_key})

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Récupérez les données de la réponse au format JSON
    data = response.json()
    # La structure des données est : [[timestamp, open, high, low, close, volume, close_time, ...], ...]
    # Nous sommes intéressés par le timestamp (index 0) et le prix de clôture (index 4) pour chaque heure
    dates = [datetime.fromtimestamp(entry[0] / 1000) for entry in data]  # Convertir timestamp en date
    prices = [float(entry[4]) for entry in data]  # Prix de clôture
    # Affichez les dates et les prix
    for date, price in zip(dates, prices):
        print(date, price)
else:
    print('Erreur lors de la récupération des données:', response.status_code)


2021-08-02 02:00:00 39147.82
2021-08-03 02:00:00 38207.05
2021-08-04 02:00:00 39723.18
2021-08-05 02:00:00 40862.46
2021-08-06 02:00:00 42836.87
2021-08-07 02:00:00 44572.54
2021-08-08 02:00:00 43794.37
2021-08-09 02:00:00 46253.4
2021-08-10 02:00:00 45584.99
2021-08-11 02:00:00 45511.0
2021-08-12 02:00:00 44399.0
2021-08-13 02:00:00 47800.0
2021-08-14 02:00:00 47068.51
2021-08-15 02:00:00 46973.82
2021-08-16 02:00:00 45901.29
2021-08-17 02:00:00 44695.95
2021-08-18 02:00:00 44705.29
2021-08-19 02:00:00 46760.62
2021-08-20 02:00:00 49322.47
2021-08-21 02:00:00 48821.87
2021-08-22 02:00:00 49239.22
2021-08-23 02:00:00 49488.85
2021-08-24 02:00:00 47674.01
2021-08-25 02:00:00 48973.32
2021-08-26 02:00:00 46843.87
2021-08-27 02:00:00 49069.9
2021-08-28 02:00:00 48895.35
2021-08-29 02:00:00 48767.83
2021-08-30 02:00:00 46982.91
2021-08-31 02:00:00 47100.89
2021-09-01 02:00:00 48810.52
2021-09-02 02:00:00 49246.64
2021-09-03 02:00:00 49999.14
2021-09-04 02:00:00 49915.64
2021-09-05 02:00:00

In [138]:
price = pd.DataFrame({'Date': dates, 'Prix': prices})


In [140]:
price['Date'] = price['Date'].dt.date


In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'],format='mixed')  # Convertir la colonne 'Date' en type datetime
df['created_at'] = df['created_at'].dt.date

In [144]:
price['Prix'] = price['Prix'].pct_change()*100


In [145]:
df.columns = ["id","main_domain","title","description","Date"]

In [146]:
merged_df = pd.merge(df, price, on='Date', how='inner')


In [149]:
final_df = merged_df[["description","Prix"]]

In [ ]:
final_df["Instruction"] = "Predict price variation in percentage of the Bictoin considering this recent market news"

In [153]:
final_df.columns =["input","output","Instruction"]

In [ ]:
df['output'] = pd.qcut(df['output'], q=4, labels=['Extreme Bearish', 'Bearish', 'Bullish', 'Extrem Bullish'])


In [154]:
final_df.to_csv('HG_Finance_Clean', index=False)
